# Beginning Steps

## Detect Encoding Using chardet

In [ ]:
import chardet
import pandas as pd

# Detect encoding
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        raw_data = f.read(100000)  # Read a sample from the file
        result = chardet.detect(raw_data)
    return result['encoding']

file_path = "/content/retail_data.csv"
detected_encoding = detect_encoding(file_path)
print(f"Detected encoding: {detected_encoding}")

# Load CSV with detected encoding
data = pd.read_csv(file_path, encoding=detected_encoding)
print(data.head())  # Display first few rows


Detected encoding: ascii
   customer_id  age  gender income_bracket loyalty_program  membership_years  \
0            1   56   Other           High              No                 0   
1            2   69  Female         Medium              No                 2   
2            3   46  Female            Low              No                 5   
3            4   32  Female            Low              No                 0   
4            5   60  Female            Low             Yes                 7   

  churned marital_status  number_of_children education_level  ...  \
0      No       Divorced                   3      Bachelor's  ...   
1      No        Married                   2             PhD  ...   
2      No        Married                   3      Bachelor's  ...   
3      No       Divorced                   2        Master's  ...   
4     Yes       Divorced                   2      Bachelor's  ...   

  distance_to_store  holiday_season  season  weekend customer_support_calls  \


## Importing Original csv File w/ correct Encoding

In [ ]:
from pandas import read_csv
import pandas as pd

# Specify the encoding as detected ('ascii')
data = pd.read_csv('/content/retail_data.csv', encoding='ascii')  # Updated encoding to 'ascii'

# Sampling to 30,000 Records

## Simple Random Sampling

In [ ]:
import pandas as pd

# Check if data has more than or equal to 20000 rows
sample_size = min(30000, len(data))  # Sample 20,000 rows

# Sample the data
sampled_data = data.sample(n=sample_size, random_state=42)

# Display the sampled data
print(sampled_data)

sampled_data.to_csv("Simple_Random_Sample.csv", index=False)

        customer_id  age  gender income_bracket loyalty_program  \
987231       987232   60  Female         Medium             Yes   
79954         79955   63   Other            Low              No   
567130       567131   72   Other         Medium             Yes   
500891       500892   34   Other           High             Yes   
55399         55400   77   Other         Medium              No   
...             ...  ...     ...            ...             ...   
493666       493667   25   Other           High              No   
356313       356314   27    Male           High             Yes   
94438         94439   20    Male            Low             Yes   
763504       763505   58  Female            Low             Yes   
940706       940707   21    Male           High              No   

        membership_years churned marital_status  number_of_children  \
987231                 8     Yes       Divorced                   3   
79954                  1     Yes         Single      

### Statistical Verification - KS-Test & Chi-Square Test

#### KS-Test (Numeric Attributes)

In [ ]:
import numpy as np
from scipy.stats import ks_2samp

# Select only numeric columns
numeric_columns = data.select_dtypes(include=np.number).columns

# Perform KS Test for each numeric column
ks_results = {}
for col in numeric_columns:
    stat, p_value = ks_2samp(data[col].dropna(), sampled_data[col].dropna())
    ks_results[col] = {'KS Statistic': stat, 'p-value': p_value}

# Convert to DataFrame for better readability
ks_df = pd.DataFrame.from_dict(ks_results, orient='index')
print("Kolmogorov-Smirnov Test Results:")
print(ks_df)

# Save KS results to CSV
ks_df.to_csv("SRS-KS_Test_Results.csv", index=True)


Kolmogorov-Smirnov Test Results:
                           KS Statistic   p-value
customer_id                    0.004593  0.568914
age                            0.003381  0.891954
membership_years               0.003544  0.856582
number_of_children             0.004757  0.523432
transaction_id                 0.003913  0.762287
product_id                     0.005213  0.405778
quantity                       0.001768  0.999988
unit_price                     0.006433  0.178569
discount_applied               0.004676  0.545683
transaction_hour               0.004747  0.526079
week_of_year                   0.006172  0.216210
month_of_year                  0.003848  0.780102
avg_purchase_value             0.004737  0.528916
avg_discount_used              0.006448  0.176617
online_purchases               0.003714  0.815194
in_store_purchases             0.003256  0.915980
avg_items_per_transaction      0.003233  0.920051
avg_transaction_value          0.007887  0.053110
total_returned_it

#### Chi-Square (Categorical Attributes)

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

# Select categorical columns
categorical_columns = data.select_dtypes(include='object').columns

chi2_results = {}

for col in categorical_columns:
    # Convert categorical values to frequency counts
    observed = data[col].value_counts().values
    expected = sampled_data[col].value_counts().reindex(data[col].unique(), fill_value=0).values

    # Perform Chi-Square Test
    chi2_stat, p_value, _, _ = chi2_contingency([observed, expected])

    # Store results
    chi2_results[col] = {'Chi-Square Statistic': chi2_stat, 'p-value': p_value}

# Convert results to a DataFrame
chi2_df = pd.DataFrame.from_dict(chi2_results, orient='index')

# Print results
print("\nChi-Square Test Results:")
print(chi2_df)

# Save results to CSV
chi2_df.to_csv("SRS-Chi_Square_Test_Results.csv", index=True)


Chi-Square Test Results:
                           Chi-Square Statistic   p-value
gender                                 0.016375  0.991846
income_bracket                         3.197016  0.202198
loyalty_program                        0.750164  0.386424
churned                                1.409705  0.235105
marital_status                         1.688061  0.429974
education_level                        4.821439  0.185349
occupation                             1.613442  0.656346
transaction_date                  498321.124167  1.000000
product_category                       9.950684  0.041267
payment_method                         4.110562  0.249770
store_location                         0.686004  0.876491
day_of_week                           11.747071  0.067855
purchase_frequency                     0.174906  0.981535
last_purchase_date                496856.492778  1.000000
preferred_store                        0.901380  0.825095
product_name                           5.65478

## 1-in-k Sampling

In [ ]:
import pandas as pd

# Define sample size
sample_size = 30000

# Compute sampling interval K
K = len(data) // sample_size

# Perform 1-in-K systematic sampling
systematic_sampled_data = data.iloc[::K, :]

# Ensure exactly 30,000 rows (adjust if needed)
systematic_sampled_data = systematic_sampled_data.head(sample_size)

# Display the sampled data
print(systematic_sampled_data)

systematic_sampled_data.to_csv("Systematic_Sample.csv", index=False)

        customer_id  age  gender income_bracket loyalty_program  \
0                 1   56   Other           High              No   
33               34   39  Female           High             Yes   
66               67   67   Other           High             Yes   
99              100   51    Male            Low             Yes   
132             133   42    Male            Low             Yes   
...             ...  ...     ...            ...             ...   
989835       989836   47  Female         Medium              No   
989868       989869   79   Other         Medium             Yes   
989901       989902   60  Female            Low              No   
989934       989935   41    Male         Medium              No   
989967       989968   77    Male         Medium              No   

        membership_years churned marital_status  number_of_children  \
0                      0      No       Divorced                   3   
33                     0     Yes       Divorced      

### Statistical Verification - KS-Test & Chi-Square Test

#### KS-Test (Numeric Attributes)

In [ ]:
import numpy as np
from scipy.stats import ks_2samp

# Select only numeric columns
numeric_columns = data.select_dtypes(include=np.number).columns

# Perform KS Test for each numeric column
ks_results = {}
for col in numeric_columns:
    stat, p_value = ks_2samp(data[col].dropna(), systematic_sampled_data[col].dropna())
    ks_results[col] = {'KS Statistic': stat, 'p-value': p_value}

# Convert to DataFrame for better readability
ks_df = pd.DataFrame.from_dict(ks_results, orient='index')
print("Kolmogorov-Smirnov Test Results:")
print(ks_df)

# Save results as CSV
ks_df.to_csv("Systematic-KS_Test_Results.csv", index=True)


Kolmogorov-Smirnov Test Results:
                           KS Statistic   p-value
customer_id                    0.010032  0.005650
age                            0.005207  0.407141
membership_years               0.004396  0.625130
number_of_children             0.001743  0.999992
transaction_id                 0.004697  0.539857
product_id                     0.004543  0.583141
quantity                       0.001833  0.999971
unit_price                     0.003539  0.857660
discount_applied               0.006002  0.243806
transaction_hour               0.002634  0.987236
week_of_year                   0.002502  0.993055
month_of_year                  0.005368  0.369562
avg_purchase_value             0.006464  0.174463
avg_discount_used              0.002549  0.991257
online_purchases               0.006073  0.231977
in_store_purchases             0.003881  0.771013
avg_items_per_transaction      0.005384  0.366019
avg_transaction_value          0.003836  0.783228
total_returned_it

#### Chi-Square (Categorical Attributes)

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

# Select categorical columns
categorical_columns = data.select_dtypes(include='object').columns

chi2_results = {}

for col in categorical_columns:
    # Convert categorical values to frequency counts
    observed = data[col].value_counts().values
    expected = systematic_sampled_data[col].value_counts().reindex(data[col].unique(), fill_value=0).values

    # Perform Chi-Square Test
    chi2_stat, p_value, _, _ = chi2_contingency([observed, expected])

    # Store results
    chi2_results[col] = {'Chi-Square Statistic': chi2_stat, 'p-value': p_value}

# Convert results to a DataFrame
chi2_df = pd.DataFrame.from_dict(chi2_results, orient='index')

# Print results
print("\nChi-Square Test Results:")
print(chi2_df)

# Save results to CSV
chi2_df.to_csv("Systematic-Chi_Square_Test_Results.csv", index=True)



Chi-Square Test Results:
                           Chi-Square Statistic   p-value
gender                                 1.721521  0.422840
income_bracket                         0.507889  0.775735
loyalty_program                        0.284072  0.594045
churned                                1.277854  0.258298
marital_status                         1.174345  0.555897
education_level                        3.384102  0.336108
occupation                             4.688463  0.196083
transaction_date                  498247.450556  1.000000
product_category                       0.254819  0.992541
payment_method                         1.507888  0.680451
store_location                         2.147803  0.542303
day_of_week                            3.720410  0.714451
purchase_frequency                     2.173643  0.537159
last_purchase_date                496901.875722  1.000000
preferred_store                        3.012489  0.389704
product_name                           5.59371

## Stratified Sampling

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Define sample size
stratified_sample_size = 30000

# Perform stratified sampling using train_test_split
stratified_sample, _ = train_test_split(
    data,
    train_size=stratified_sample_size,
    stratify=data['income_bracket'],
    random_state=42
)

# Display sampled data
print(stratified_sample.head())

stratified_sample.to_csv("Stratified_Sample.csv", index=False)

        customer_id  age  gender income_bracket loyalty_program  \
862287       862288   42   Other           High              No   
980510       980511   53  Female            Low              No   
768782       768783   66   Other            Low             Yes   
874826       874827   64    Male         Medium              No   
476042       476043   59  Female           High              No   

        membership_years churned marital_status  number_of_children  \
862287                 3     Yes       Divorced                   4   
980510                 5     Yes       Divorced                   0   
768782                 0      No       Divorced                   2   
874826                 0      No        Married                   0   
476042                 8     Yes        Married                   3   

       education_level  ... distance_to_store  holiday_season  season  \
862287      Bachelor's  ...             45.18             Yes  Spring   
980510     High School  

### Statistical Verification - KS-Test & Chi-Square Test

#### KS-Test (Numeric Attributes)

In [ ]:
import numpy as np
from scipy.stats import ks_2samp

# Select numeric attributes
numeric_features = data.select_dtypes(include=np.number).columns

# Perform KS test for each numeric attribute
ks_test_output = {}
for feature in numeric_features:
    ks_statistic, ks_pvalue = ks_2samp(data[feature].dropna(), stratified_sample[feature].dropna())
    ks_test_output[feature] = {'KS Statistic': ks_statistic, 'p-value': ks_pvalue}

# Convert results to DataFrame
ks_results_df = pd.DataFrame.from_dict(ks_test_output, orient='index')
print("Kolmogorov-Smirnov Test Results (Stratified Sampling):")
print(ks_results_df)

# Save KS results to CSV
ks_results_df.to_csv("Stratified-KS_Test_Results.csv", index=True)


Kolmogorov-Smirnov Test Results (Stratified Sampling):
                           KS Statistic   p-value
customer_id                    0.004267  0.662244
age                            0.005384  0.366019
membership_years               0.005289  0.387682
number_of_children             0.003490  0.868694
transaction_id                 0.008864  0.020445
product_id                     0.004269  0.661669
quantity                       0.002765  0.978714
unit_price                     0.005284  0.389008
discount_applied               0.004813  0.508290
transaction_hour               0.003319  0.904284
week_of_year                   0.003264  0.914478
month_of_year                  0.002165  0.999154
avg_purchase_value             0.003671  0.826029
avg_discount_used              0.004282  0.657927
online_purchases               0.003996  0.739315
in_store_purchases             0.005470  0.346884
avg_items_per_transaction      0.006663  0.149851
avg_transaction_value          0.004972  0.46

#### Chi-Square (Categorical Attributes)

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

# Select categorical attributes
categorical_features = data.select_dtypes(include='object').columns

chi_square_output = {}

for feature in categorical_features:
    # Get frequency counts for observed and expected distributions
    observed = data[feature].value_counts().values
    expected = stratified_sample[feature].value_counts().reindex(data[feature].unique(), fill_value=0).values

    # Perform Chi-Square Test
    chi_statistic, chi_pvalue, _, _ = chi2_contingency([observed, expected])

    # Store results
    chi_square_output[feature] = {'Chi-Square Statistic': chi_statistic, 'p-value': chi_pvalue}

# Convert results to DataFrame
chi_results_df = pd.DataFrame.from_dict(chi_square_output, orient='index')

# Print results
print("\nChi-Square Test Results (Stratified Sampling):")
print(chi_results_df)

# Save results to CSV
chi_results_df.to_csv("Stratified-Chi_Square_Test_Results.csv", index=True)


Chi-Square Test Results (Stratified Sampling):
                           Chi-Square Statistic   p-value
gender                                 1.875268  0.391553
income_bracket                         0.000026  0.999987
loyalty_program                        0.000092  0.992329
churned                                1.382817  0.239622
marital_status                         0.719212  0.697951
education_level                        3.124317  0.372852
occupation                             2.018005  0.568678
transaction_date                  498185.564722  1.000000
product_category                       0.833767  0.933865
payment_method                         7.267311  0.063849
store_location                         2.368688  0.499490
day_of_week                            3.820662  0.700931
purchase_frequency                     4.202355  0.240426
last_purchase_date                496829.970278  1.000000
preferred_store                        1.673182  0.642912
product_name            

# EDA

## Generating Profiling Reports for Each Dataset

### Full Dataset

In [ ]:
'''
import pandas as pd
from ydata_profiling import ProfileReport
import gc  # Garbage collection to free memory

full_data = pd.read_csv("retail_data.csv")

# Generate Simple Random Sample Report
full_data = ProfileReport(full_data, title="Full Dataset")
full_data.to_file("full_data.html")
del srs_report
gc.collect()
'''

'\nimport pandas as pd\nfrom ydata_profiling import ProfileReport\nimport gc  # Garbage collection to free memory\n\nfull_data = pd.read_csv("retail_data.csv")\n\n# Generate Simple Random Sample Report\nfull_data = ProfileReport(full_data, title="Full Dataset")\nfull_data.to_file("full_data.html")\ndel srs_report\ngc.collect()\n'

### Simple Random Sample (S.R.S)

In [ ]:
'''
import pandas as pd
from ydata_profiling import ProfileReport
import gc  # Garbage collection to free memory

srs_sample = pd.read_csv("Simple_Random_Sample.csv")

# Generate Simple Random Sample Report
srs_report = ProfileReport(srs_sample, title="Simple Random Sample")
srs_report.to_file("SRS_Report.html")
del srs_report
gc.collect()
'''

'\nimport pandas as pd\nfrom ydata_profiling import ProfileReport\nimport gc  # Garbage collection to free memory\n\nsrs_sample = pd.read_csv("Simple_Random_Sample.csv")\n\n# Generate Simple Random Sample Report\nsrs_report = ProfileReport(srs_sample, title="Simple Random Sample")\nsrs_report.to_file("SRS_Report.html")\ndel srs_report\ngc.collect()\n'

### Systematic

In [ ]:
'''
!pip install ydata-profiling

import pandas as pd
from ydata_profiling import ProfileReport
import gc  # Garbage collection to free memory

systematic_sample = pd.read_csv("Systematic_Sample.csv")

# Generate Systematic Sample Report
systematic_report = ProfileReport(systematic_sample, title="Systematic Sample")
systematic_report.to_file("Systematic_Report.html")
del systematic_report
gc.collect()
'''

'\n!pip install ydata-profiling\n\nimport pandas as pd\nfrom ydata_profiling import ProfileReport\nimport gc  # Garbage collection to free memory\n\nsystematic_sample = pd.read_csv("Systematic_Sample.csv")\n\n# Generate Systematic Sample Report\nsystematic_report = ProfileReport(systematic_sample, title="Systematic Sample")\nsystematic_report.to_file("Systematic_Report.html")\ndel systematic_report\ngc.collect()\n'

### Stratified

In [ ]:
'''
import pandas as pd
from ydata_profiling import ProfileReport
import gc  # Garbage collection to free memory

stratified_sample = pd.read_csv("Stratified_Sample.csv")

# Generate Stratified Sample Report
stratified_report = ProfileReport(stratified_sample, title="Stratified Sample")
stratified_report.to_file("Stratified_Report.html")
del stratified_report
gc.collect()
'''

'\nimport pandas as pd\nfrom ydata_profiling import ProfileReport\nimport gc  # Garbage collection to free memory\n\nstratified_sample = pd.read_csv("Stratified_Sample.csv")\n\n# Generate Stratified Sample Report\nstratified_report = ProfileReport(stratified_sample, title="Stratified Sample")\nstratified_report.to_file("Stratified_Report.html")\ndel stratified_report\ngc.collect()\n'

## Comparing Full Dataset vs. Each Sampled Dataset

### Full Dataset vs. Simple Random Sample (S.R.S)

In [ ]:
import pandas as pd
from ydata_profiling import ProfileReport
import gc

# Load datasets
full_data = pd.read_csv("retail_data.csv")
srs_sample = pd.read_csv("Simple_Random_Sample.csv")

# Generate minimal ProfileReports
full_report = ProfileReport(full_data, title="Full Dataset", minimal=True)
srs_report = ProfileReport(srs_sample, title="Simple Random Sample", minimal=True)

# Generate comparison report
comparison_srs = full_report.compare(srs_report)
comparison_srs.to_file("Full_vs_SRS_Comparison.html")

# Clean up memory
del full_data, srs_sample, full_report, srs_report, comparison_srs
gc.collect()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 78/78 [01:53<00:00,  1.46s/it]


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 78/78 [00:03<00:00, 22.16it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

76

### Full Dataset vs. Systematic

In [ ]:
import pandas as pd
from ydata_profiling import ProfileReport
import gc

# Load datasets
full_data = pd.read_csv("retail_data.csv")
systematic_sample = pd.read_csv("Systematic_Sample.csv")

# Generate minimal ProfileReports
full_report = ProfileReport(full_data, title="Full Dataset", minimal=True)
systematic_report = ProfileReport(systematic_sample, title="Systematic Sample", minimal=True)

# Generate comparison report
comparison_systematic = full_report.compare(systematic_report)
comparison_systematic.to_file("Full_vs_Systematic_Comparison.html")

# Clean up memory
del full_data, systematic_sample, full_report, systematic_report, comparison_systematic
gc.collect()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 78/78 [01:52<00:00,  1.44s/it]


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 78/78 [00:04<00:00, 18.29it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

79

### Full Dataset vs. Stratified

In [ ]:
import pandas as pd
from ydata_profiling import ProfileReport
import gc

# Load datasets
full_data = pd.read_csv("retail_data.csv")
stratified_sample = pd.read_csv("Stratified_Sample.csv")

# Generate minimal ProfileReports
full_report = ProfileReport(full_data, title="Full Dataset", minimal=True)
stratified_report = ProfileReport(stratified_sample, title="Stratified Sample", minimal=True)

# Generate comparison report
comparison_stratified = full_report.compare(stratified_report)
comparison_stratified.to_file("Full_vs_Stratified_Comparison.html")

# Clean up memory
del full_data, stratified_sample, full_report, stratified_report, comparison_stratified
gc.collect()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 78/78 [01:59<00:00,  1.53s/it]


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 78/78 [00:04<00:00, 16.37it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

117